In [26]:
import arcpy   
import numpy as np
from osgeo import gdal
import os
import imageio           
import glob  
import shutil

# 1

In [11]:
def ClipFromFolder(raster,inputfolder,outputfolder):
    """从一个文件夹中读取栅格以裁剪另外一个栅格

    Args:
        raster (栅格):待裁剪栅格
        inputfolder (输入文件夹): 包含作为裁剪要素的范围栅格
        outputfolder (输出文件夹): 输出目录
    """
    arcpy.env.workspace = inputfolder
    arcpy.env.overwriteOutput = True
    l = arcpy.ListRasters()
    for i in range(len(l)):
        outraster = outputfolder + "\\" + str(i) + ".tif"
        description = arcpy.Describe(l[i])
        extent = str(description.extent.XMin) + " " + str(description.extent.YMin) + " " + str(description.extent.XMax) + " " + str(description.extent.YMax)
        arcpy.management.Clip(raster, extent, outraster, l[i], "255", "NONE", "MAINTAIN_EXTENT")
        print( str(i) + ".tif done!")

# 2

In [24]:
def movefile(RefFloder,OriFloder,DesFloder):
    """移动文件，参考RefFloder中的文件，从OriFloder中找到同名文件，移动到DesFloder中
    Args:
        RefFloder (_type_): _description_
        OriFloder (_type_): _description_
        DesFloder (_type_): _description_
    """
    arcpy.env.workspace = RefFloder
    l = arcpy.ListRasters()
    for i in range(len(l)):
        shutil.move(OriFloder + '/' + l[i], DesFloder + "\\" + l[i])

# 3

In [12]:
def Normalization(inputfolder,outputfolder,upper):
    oldpath = inputfolder
    newpath = outputfolder
    arcpy.env.workspace = oldpath
    l = arcpy.ListRasters()
    for i in range(len(l)):
        R = arcpy.Raster(oldpath + "\\" + l[i])
        arcpy.CalculateStatistics_management(oldpath + "\\" + l[i])
        k = upper / (R.maximum - R.minimum + 10)
        R1 = k * (R - R.minimum)
        R1.save(newpath + "\\" + l[i])
        print(l[i] + "done!")

# 4

In [ ]:
DataFolder = r"D:\proproject\样本制作\ALLData\data"
OtherDataNames = ["C_山体阴影","C_影像","C_Slope"] 
def CompositeBands(DataFolder,OtherDataNames,OutPath):
    """生成多波段影像
        数据类型数量改变时需要更改函数。
    Args:
        DataFolder (_type_): _description_
        OtherDataNames (_type_): _description_
        OutPath (_type_): _description_
    """
    DEM = DataFolder + "\\DEM"
    arcpy.env.workspace = DEM
    l = arcpy.ListRasters()
    names = [j for j in OtherDataNames]
    for i in range(len(l)):
        R = []
        DEM = DataFolder + "\\" + names[0] + "\\" + l[i] 
        Hillshade = DataFolder + "\\" + names[1] + "\\" + l[i]
        Image = DataFolder + "\\" + names[2] + "\\" + l[i]
        Slope = DataFolder + "\\" + names[3] + "\\" + l[i]
        R.append(DEM)
        R.append(Hillshade)
        R.append(Image)
        R.append(Slope)
        if os.path.exists(OutPath + "\\" + l[i]) == False:
            arcpy.CompositeBands_management(R, OutPath + "\\" + l[i])

In [13]:
def readtiff2array(oriPath):
    in_ds = gdal.Open(oriPath)
    print("open tif file succeed")

    # 读取原图中的每个波段
    row = in_ds.RasterYSize  # 行
    col = in_ds.RasterXSize  # 列
    band = in_ds.RasterCount  # 波段
    geoTrans = in_ds.GetGeoTransform()
    geoPro = in_ds.GetProjection()
    
    nodatavalue = in_ds.GetRasterBand(1).GetNoDataValue()

    # specific datatype
#     newDataType = gdal.GDT_UInt16;
#     data = np.zeros([row, col, band], newDataType)  # 建立数组保存读取的tiff

    
    # according to the input datatype
    datatype_index = in_ds.GetRasterBand(1).DataType
    datatype = gdal.GetDataTypeName(datatype_index)
#     print(datatype)
    if 'GDT_Byte' in datatype:
        newDataType = 'int8'
    elif 'GDT_UInt16' in datatype:
        newDataType = 'int16'
    else:
        newDataType = 'float32'
    data = np.zeros([row, col, band], newDataType)  # 建立数组保存读取的tiff

    for i in range(band):
        dt = in_ds.GetRasterBand(i + 1)
        # 从每个波段中裁剪需要的矩形框内的数据
        data[:, :, i] = dt.ReadAsArray(0, 0, col, row)             #dataset.ReadAsArray(xoff=0, yoff=0, xsize=512, ysize=512)，前两个参数为该坐标与左上角的像素的相对位置

    del in_ds
    
    return data, [band, datatype, geoTrans, geoPro], nodatavalue
    # data shape = HWC

# 5

In [14]:
def transfer_16bit_to_8bit(data, isSlope=False, isDEM=False, nodatavalue=91):
    # transform to 8bit

    
    # 以下两部分主要是为了处理无效值，要将无效值区域调至max
    
    ## slope 数据中无效值被设置为了91
    if isSlope is True:
        
        min_16bit = 50000
        max_16bit = 0
    
        # 先对有效值区域进行处理，找到有效值的最大最小值
        ## 找到有效值的所有索引
        indexLS = np.argwhere(data<nodatavalue)
        ## indexLS为tuple，shape=[n,3],n代表有多少行 
        ## 具体在图像中的坐标为 indexLS.shape[0][0], indexLS.shape[0][1]
        rows = indexLS.shape[0]
        i = 0
        for i in range(rows):
            ## 找到其临时值
            tempValue = data[indexLS[i][0],indexLS[i][1],0]
            ## 对临时值进行判断
            if tempValue != 91:
                if tempValue > max_16bit:
                    max_16bit = tempValue
                elif tempValue < min_16bit:
                    min_16bit = tempValue      

        # 进而对无效值（通常为91）进行处理，思路为将其改为最大值+最大值的10%
        # 循环过程与上一步类似
        indexGE = np.argwhere(data>=nodatavalue)    
        rows = indexGE.shape[0]
        i = 0
        for i in range(rows):
            data[indexGE[i][0],indexGE[i][1],0] = max_16bit #+ max_16bit*0.1
        print(max_16bit, min_16bit)
        
    ## dem数据中无效值被设置为了0
    elif isDEM is True:
        nodatavalue = 0
        min_16bit = 50000
        max_16bit = 0
        
        # 先对有效值区域进行处理，找到有效值的最大最小值
        ## 找到有效值的所有索引
        indexGS = np.argwhere(data>nodatavalue)
        ## indexGS为tuple，shape=[n,3],n代表有多少行 
        ## 具体在图像中的坐标为 indexLS.shape[0][0], indexLS.shape[0][1]
        rows = indexGS.shape[0]
        i = 0
        for i in range(rows):
            ## 找到其临时值
            tempValue = data[indexGS[i][0],indexGS[i][1],0]
            ## 对临时值进行判断
            if tempValue != 0:
                if tempValue > max_16bit:
                    max_16bit = tempValue
                elif tempValue < min_16bit:
                    min_16bit = tempValue      

        # 进而对无效值（通常为0）进行处理，思路为将其改为最大值+最大值的10%
        # 循环过程与上一步类似
        indexLE = np.argwhere(data<=nodatavalue)    
        rows = indexLE.shape[0]
        i = 0
        for i in range(rows):
            data[indexLE[i][0],indexLE[i][1],0] = max_16bit #+ max_16bit*0.1
        print(max_16bit, min_16bit)
    else:   
        min_16bit = np.min(data)
        max_16bit = np.max(data)
    
    data_8bit = np.array(np.rint(255 * ((data - min_16bit) / (max_16bit - min_16bit))), dtype=np.uint8)                   #np.rint四舍五入
    return data_8bit

In [15]:
def calculateTransform(ori_transform, offsetX, offsetY):
    # 读取原图仿射变换参数值
    top_left_x = ori_transform[0]  # 左上角x坐标
    w_e_pixel_resolution = ori_transform[1]  # 东西方向像素分辨率
    top_left_y = ori_transform[3]  # 左上角y坐标
    n_s_pixel_resolution = ori_transform[5]  # 南北方向像素分辨率

    # 根据反射变换参数计算新图的原点坐标
    top_left_x = top_left_x + offsetX * w_e_pixel_resolution
    top_left_y = top_left_y + offsetY * n_s_pixel_resolution

    # 将计算后的值组装为一个元组，以方便设置
    dst_transform = (top_left_x, ori_transform[1], ori_transform[2], top_left_y, ori_transform[4], ori_transform[5])

    return dst_transform

In [16]:
def cliptopatch(inputData, ori_geoTrans, ori_geoPro, saveName, offsetCor, patchSize, nodatavalue):
    if os.path.exists(saveName) == False:
        # 读取输入数据信息
        ori_Datatype = inputData.dtype
        # inputData
        if len(inputData.shape) == 3:
            in_bands = inputData.shape[2]
        else:
            in_bands = 1

        # 读取要裁剪的原图
        out_band = np.zeros([patchSize, patchSize, in_bands], ori_Datatype)
        # print(out_band.shape)
        for i in range(in_bands):
            out_band[:, :, i] = inputData[offsetCor[0]:offsetCor[0] + patchSize, offsetCor[1]:offsetCor[1] + patchSize, i]
            out_band[:, :, i] = np.where(out_band[:, :, i]==nodatavalue,-1,out_band[:, :, i])
            
        # 获取原图的原点坐标信息
        ori_transform = ori_geoTrans
        # 计算仿射变化参数
        dst_transform = calculateTransform(ori_transform, offsetCor[1], offsetCor[0])
        # 设置DataType
        if 'int8' in out_band.dtype.name:
            newDataType = gdal.GDT_Byte
        elif 'int16' in out_band.dtype.name:
            newDataType = gdal.GDT_UInt16
        else:
            newDataType = gdal.GDT_Float32

        # 创建gtiff 并 写入
        driver = gdal.GetDriverByName("GTiff")
        
        if not np.any(out_band[:, :, i] == -1) :             #Nodata是-1
            dataset = driver.Create(saveName, patchSize, patchSize, in_bands, newDataType)
            if (dataset != None):
                dataset.SetGeoTransform(dst_transform)  # 写入仿射变换参数
                dataset.SetProjection(ori_geoPro)  # 写入投影
            for i in range(in_bands):
                dataset.GetRasterBand(i + 1).WriteArray(out_band[:, :, i])
            del dataset


In [17]:
def clipgeotiff(inputPath, savePath, patchSize, patchIntersection, startCol, startRow, typeName, index, To8bit=False, isSlope=False,isDEM=False):
    os.makedirs(savePath, exist_ok=True)
    # 遍历文件夹
    from_names = glob.glob(os.path.join(inputPath, "*.tif"))

    # 左上角坐标移动步长
    stride = patchSize - int((patchSize - patchIntersection) / 2)
    print(from_names)
    for i in range(len(from_names)):
        print(from_names[i])
        filePath = os.path.join(inputPath, os.path.basename(from_names[i]))
        # oriPara=[band, datatype, geoTrans, geoPro]
        inputData, oriPara, nodatavalue = readtiff2array(filePath)
        
        # 16bit to 8bit
        if To8bit is True:
            inputData = transfer_16bit_to_8bit(inputData, isSlope,isDEM)

        # 裁剪 起始坐标
        # 原始代码的[offsetY, offsetX] = [offsetRow, offsetCol]
        offsetCor = [startRow, startCol]
        while offsetCor[1] <= inputData.shape[1] - patchSize:
            while offsetCor[0] <= inputData.shape[0] - patchSize:
                filename = typeName + str(index) + '_' + str(offsetCor[1]) + '_' + str(offsetCor[0]) + '.tif'
                saveName = os.path.join(savePath, filename)
                cliptopatch(inputData, oriPara[2], oriPara[3], saveName, offsetCor, patchSize, nodatavalue)
                offsetCor[0] = offsetCor[0] + stride
            offsetCor[1] = offsetCor[1] + stride
            offsetCor[0] = 0
        index = index + 1

In [29]:
#从文件夹A中读取栅格以裁剪另外一个栅格，输出到文件夹B中，再将B中的栅格归一化到0-upper储存再文件夹C中，最后将文件夹C中的栅格裁剪成小块到文件夹D中。

#待裁剪栅格
Raster = r"D:\proproject\样本制作\ALLData\未裁剪数据\slope10.tif"   
#范围栅格所在文件夹
Folder_A = r"D:\proproject\样本制作\ALLData\data\DEM"   
#裁剪输出                 
Folder_B = r"D:\proproject\样本制作\ALLData\data\坡度"
#归一化输出
Folder_C = r"D:\proproject\样本制作\ALLData\data\坡度0-255"
#裁剪成小块输出
Folder_D = r"D:\proproject\样本制作\ALLData\data\C_Slope"
upper = 255
ClipFromFolder(Raster,Folder_A,Folder_B)
Normalization(Folder_B,Folder_C,upper)
patchSize = 224
patchIntersection = 200
startCol = 0
startRow = 0
typeName = 'W'
index = 0
clipgeotiff(Folder_C, Folder_D, patchSize, patchIntersection, startCol, startRow, typeName, index, To8bit=False, isSlope=False,isDEM=True)

0.tif done!
1.tif done!
2.tif done!
3.tif done!
4.tif done!
5.tif done!
6.tif done!
7.tif done!
8.tif done!
9.tif done!
10.tif done!
11.tif done!
12.tif done!
13.tif done!
14.tif done!
15.tif done!
16.tif done!
17.tif done!
18.tif done!
19.tif done!
20.tif done!
21.tif done!
22.tif done!
23.tif done!
0.tifdone!
1.tifdone!
10.tifdone!
11.tifdone!
12.tifdone!
13.tifdone!
14.tifdone!
15.tifdone!
16.tifdone!
17.tifdone!
18.tifdone!
19.tifdone!
2.tifdone!
20.tifdone!
21.tifdone!
22.tifdone!
23.tifdone!
3.tifdone!
4.tifdone!
5.tifdone!
6.tifdone!
7.tifdone!
8.tifdone!
9.tifdone!
['D:\\proproject\\样本制作\\ALLData\\data\\坡度0-255\\0.tif', 'D:\\proproject\\样本制作\\ALLData\\data\\坡度0-255\\1.tif', 'D:\\proproject\\样本制作\\ALLData\\data\\坡度0-255\\10.tif', 'D:\\proproject\\样本制作\\ALLData\\data\\坡度0-255\\11.tif', 'D:\\proproject\\样本制作\\ALLData\\data\\坡度0-255\\12.tif', 'D:\\proproject\\样本制作\\ALLData\\data\\坡度0-255\\13.tif', 'D:\\proproject\\样本制作\\ALLData\\data\\坡度0-255\\14.tif', 'D:\\proproject\\样本制作\\ALLDat

In [30]:
RefFloder = r"D:\proproject\样本制作\ALLData\Testing_data\DEM"
OriFloder = r"D:\proproject\样本制作\ALLData\data\C_Slope"
DesFloder = r"D:\proproject\样本制作\ALLData\Testing_data\Slope"
movefile(RefFloder,OriFloder,DesFloder)

In [31]:
OtherDataNames = ["C_山体阴影","C_影像","C_Slope"] 
names = [i for i in OtherDataNames]

In [32]:
names

['C_山体阴影', 'C_影像', 'C_Slope']